In [1]:
import pandas as pd
import time
import threading
from kafkaconsumer import Kafka_Consumer

In [ ]:
def get_env_params():
    return pd.read_csv('./env_params/env_params.csv',index_col=0, header=None).squeeze().to_dict()

In [2]:
if __name__ == "__main__":
    
    params = get_env_params()

    #object1 
    kafkaconsumer1 = Kafka_Consumer(params['consumer_topic1'],[params['bootstrapservers']])
    kafkaconsumer2 = Kafka_Consumer(params['consumer_topic2'],[params['bootstrapservers']])

    
    
    # Create threads for each function
    thread1 = threading.Thread(target=kafkaconsumer1.consume_and_upload
                               ,args=(params['adls_account_name'], params['adls_sas_token']
                                      ,int(params['kafkabatchsize']),params['adls_container']))
    thread2 = threading.Thread(target=kafkaconsumer2.consume_and_upload,args=(params['adls_account_name'], params['adls_sas_token']
                                      ,int(params['kafkabatchsize']),params['adls_container']))
    

    # Start the threads
    thread1.start()
    thread2.start()

    # Wait for both threads to complete
    thread1.join()
    thread2.join()